In [1]:
import pandas as pd
import numpy as np


In [10]:

# IMPORT DATA
loyalty_df = pd.read_csv("../data/receipt_item-2025010915-fe8603095238452290775091f1eabc2f.csv", skiprows=1)
# transctional_df=pd.read_csv('../data/2023 to 2024 Corp by stores ticket level.csv')


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_50489/1899899303.py:2: DtypeWarning: Columns (1,10,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  loyalty_df = pd.read_csv("../data/receipt_item-2025010915-fe8603095238452290775091f1eabc2f.csv", skiprows=1)


In [3]:
loyalty_df.columns


Index(['Transaction Number', 'Bar Code', 'Guest ID', 'Location ID',
       'Checkin ID', 'Receipt DateTime', 'Revenue ID', 'Revenue Code',
       'Location Name', 'Store Number', 'Menu Item ID', 'Menu Item Name',
       'Amount', 'Menu Item Type', 'Menu Item Family', 'Menu Item Group',
       'Quantity', 'Rate'],
      dtype='object')

In [4]:
# LOYALTY PREPROCESS


datetime_col=['Receipt DateTime']
float_col=['Menu Item ID']

loyalty_df[datetime_col] = loyalty_df[datetime_col].apply(pd.to_datetime, errors='coerce')
loyalty_df[float_col] =loyalty_df[float_col] .apply(pd.to_numeric, errors='coerce')
loyalty_df['date']=loyalty_df['Receipt DateTime'].dt.date



loyalty_df['StoreId'] = loyalty_df['Location Name'].str[-4:]
loyalty_df['StoreId']=loyalty_df['StoreId'].astype(float)

loyalty_df['time'] = np.where(loyalty_df['StoreId'].isin([1051,1052]),loyalty_df['Receipt DateTime'] + pd.Timedelta(hours=1),loyalty_df['Receipt DateTime'])
loyalty_df['time']=pd.to_datetime(loyalty_df['time'])

loyalty_df['time'] = loyalty_df['time'].dt.time



loyalty_df=loyalty_df[loyalty_df['Menu Item Type']=='M']

loyalty_df=loyalty_df[loyalty_df['StoreId'].isin([1001, 1003, 1005, 1006, 1008, 1051,1052])]


# # Concatenate 'StoreID', 'Date', and 'CheckTime' to create 'Key'
loyalty_df['Key'] = loyalty_df['StoreId'].astype(str) + "_" + loyalty_df['date'].astype(str) + "_" + loyalty_df['time'].astype(str)
# MAKING SUMMARY TABLE FROM LOYALTY TABLE
summary_loyal=loyalty_df.groupby(['Key', 'Checkin ID'])['Menu Item ID'].unique().reset_index()





Index(['StoreId', 'Date', 'CheckTime', 'UniqueCheckId', 'PLU', 'MenuItemId',
       'MenuItemName', 'ItemCount', 'Amount', 'Tax', 'LineType',
       'LineTypeDescription', 'CategoryName', 'ReductionId', 'ReductionName',
       'order_channel', 'Order_type', 'discount_amt', 'DiscountName'],
      dtype='object')

In [24]:
# LOYALTY PREPROCESS

loyalty_df = pd.read_csv("../data/receipt_item-2025010915-fe8603095238452290775091f1eabc2f.csv", skiprows=1)

datetime_col=['Receipt DateTime']
float_col=['Menu Item ID','Checkin ID']

loyalty_df[datetime_col] = loyalty_df[datetime_col].apply(pd.to_datetime, errors='coerce')
loyalty_df[float_col] =loyalty_df[float_col] .apply(pd.to_numeric, errors='coerce')
loyalty_df['date']=loyalty_df['Receipt DateTime'].dt.date

loyalty_df=loyalty_df[loyalty_df['Location Name']!='(CLOSED) Windsor (Royal Vista Cir)']
loyalty_df['StoreId'] = loyalty_df['Location Name'].str[-4:].astype(float)



# Adjust time for specific StoreIds
loyalty_df.loc[loyalty_df['StoreId'].isin({1051, 1052}), 'Receipt DateTime'] += pd.Timedelta(hours=1)
loyalty_df['time']=pd.to_datetime(loyalty_df['time'])
loyalty_df['time'] = loyalty_df['time'].dt.time



loyalty_df=loyalty_df[(loyalty_df['Menu Item Type']=='M')& (loyalty_df['StoreId'].isin([1001, 1003, 1005, 1006, 1008, 1051,1052]))]

# # Concatenate 'StoreID', 'Date', and 'CheckTime' to create 'Key'
loyalty_df['Key'] = loyalty_df['StoreId'].astype(str) + "_" + loyalty_df['date'].astype(str) + "_" + loyalty_df['time'].astype(str)
# MAKING SUMMARY TABLE FROM LOYALTY TABLE
summary_loyal=loyalty_df.groupby(['Key', 'Checkin ID'])['Menu Item ID'].unique().reset_index()

summary_loyal.head()


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_50489/1783586752.py:3: DtypeWarning: Columns (1,10,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  loyalty_df = pd.read_csv("../data/receipt_item-2025010915-fe8603095238452290775091f1eabc2f.csv", skiprows=1)
/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_50489/1783586752.py:8: FutureWarning: Parsed string "January 01, 2023 09:58 AM MST" included an un-recognized timezone "MST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  loyalty_df[datetime_col] = loyalty_df[datetime_col].apply(pd.to_datetime, errors='coerce')


KeyError: 'time'

In [22]:
import pandas as pd
import numpy as np

# Load data efficiently
loyalty_df = pd.read_csv(
    "../data/receipt_item-2025010915-fe8603095238452290775091f1eabc2f.csv",
    skiprows=1,
# Define dtypes upfront
    parse_dates=['Receipt DateTime']  # Directly parse datetime
)

# Extract date efficiently
loyalty_df['date'] = loyalty_df['Receipt DateTime'].dt.date

# Filter out closed locations before processing
loyalty_df=loyalty_df[loyalty_df['Location Name']!='(CLOSED) Windsor (Royal Vista Cir)']


# Extract StoreId from the last 4 digits of 'Location Name'
loyalty_df['StoreId'] = loyalty_df['Location Name'].str[-4:].astype(float)

# Adjust time for specific StoreIds
loyalty_df.loc[loyalty_df['StoreId'].isin({1051, 1052}), 'Receipt DateTime'] += pd.Timedelta(hours=1)

# Extract time part after adjustment
loyalty_df['time'] = loyalty_df['Receipt DateTime'].dt.time

# Filter valid Menu Item Types and Store IDs
valid_store_ids = {1001, 1003, 1005, 1006, 1008, 1051, 1052}
loyalty_df = loyalty_df[(loyalty_df['Menu Item Type'] == 'M') & (loyalty_df['StoreId'].isin(valid_store_ids))]

# Create 'Key' efficiently
loyalty_df['Key'] = loyalty_df['StoreId'].astype(int).astype(str) + "_" + loyalty_df['date'].astype(str) + "_" + loyalty_df['time'].astype(str)

# Create summary table
summary_loyal = loyalty_df.groupby(['Key', 'Checkin ID'])['Menu Item ID'].unique().reset_index()

summary_loyal.head()


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_50489/4272945602.py:5: DtypeWarning: Columns (1,10,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  loyalty_df = pd.read_csv(
/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_50489/4272945602.py:5: FutureWarning: Parsed string "January 01, 2023 09:58 AM MST" included an un-recognized timezone "MST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  loyalty_df = pd.read_csv(


,Key,Checkin ID,Menu Item ID
0,1001_2023-01-01_11:56:00,856078619,"[1100, 600, 1830, 8450, 7040]"
1,1001_2023-01-03_17:19:00,858899183,"[1510, 810, 1840, 8253, 7061, 8163, 2000, 8450]"
2,1001_2023-01-03_17:20:00,858899802,"[9010, 1800, 1840, 8450, 7061, 2000]"
3,1001_2023-01-04_08:21:00,859153476,"[1960, 9510, 7983, 7010]"
4,1001_2023-01-04_10:37:00,859240122,"[12550, 8400, 7051, 2000, 8450]"


In [33]:
# LOYALTY PREPROCESS


datetime_col=['Receipt DateTime']

float_col=['Menu Item ID','Quantity','Rate','Transaction Number',
'Bar Code','Guest ID','Location ID','Checkin ID','Revenue ID',
'Store Number']
# float_col=['Menu Item ID']

loyalty_df[datetime_col] = loyalty_df[datetime_col].apply(pd.to_datetime, errors='coerce')
loyalty_df[float_col] =loyalty_df[float_col] .apply(pd.to_numeric, errors='coerce')
loyalty_df['date']=loyalty_df['Receipt DateTime'].dt.date
# loyalty_df['time']=loyalty_df['Receipt DateTime'].dt.time


# loyalty_df=loyalty_df[loyalty_df['Location Name']!='(CLOSED) Windsor (Royal Vista Cir)']

loyalty_df['StoreId'] = loyalty_df['Location Name'].str[-4:]
loyalty_df['StoreId']=loyalty_df['StoreId'].astype(float)

# Adjust time for specific StoreIds
loyalty_df.loc[loyalty_df['StoreId'].isin({1051, 1052}), 'Receipt DateTime'] += pd.Timedelta(hours=1)

# loyalty_df['hour']=loyalty_df['time'].dt.hour
loyalty_df['time'] = loyalty_df['time'].dt.time
# loyalty_df['month']=loyalty_df['Receipt DateTime'].dt.month
# loyalty_df['year']=loyalty_df['Receipt DateTime'].dt.year


loyalty_df=loyalty_df[loyalty_df['Menu Item Type']=='M']
# # store_ids=loyalty_df['StoreId'].unique()
# print(loyalty_df['StoreId'].unique())
loyalty_df=loyalty_df[loyalty_df['StoreId'].isin([1001, 1003, 1005, 1006, 1008, 1051,1052])]
# print(loyalty_df['StoreId'].unique())

# # Concatenate 'StoreID', 'Date', and 'CheckTime' to create 'Key'
loyalty_df['Key'] = loyalty_df['StoreId'].astype(str) + "_" + loyalty_df['date'].astype(str) + "_" + loyalty_df['time'].astype(str)



/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_837/142567062.py:9: FutureWarning: Parsed string "January 01, 2023 09:58 AM MST" included an un-recognized timezone "MST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  loyalty_df[datetime_col] = loyalty_df[datetime_col].apply(pd.to_datetime, errors='coerce')


[1008. 1031. 1006. 1003. 1052. 1005. 2609. 1001. 2620. 1036. 1051. 2616.]
[1008. 1006. 1003. 1052. 1005. 1001. 1051.]


In [34]:
# MAKING SUMMARY TABLE FROM LOYALTY TABLE
summary_loyal=loyalty_df.groupby(['Key', 'Checkin ID'])['Menu Item ID'].unique().reset_index()



In [8]:
# TRANSACTIONAL DATA PRE_PROCESS

datetime_col=['Date','CheckTime']
float_col=['StoreId','UniqueCheckId','MenuItemId','PLU']



transctional_df[datetime_col] = transctional_df[datetime_col].apply(pd.to_datetime, errors='coerce')
transctional_df[float_col] = transctional_df[float_col] .apply(pd.to_numeric, errors='coerce')

transctional_df['date']=transctional_df['Date'].dt.date
# transctional_df['time']=transctional_df['CheckTime'].dt.time


# Add one hour
# transctional_df['time'] = transctional_df['CheckTime'] + pd.Timedelta(hours=1)
# transctional_df['hour']=transctional_df['Date'].dt.hour
# transctional_df['month']=transctional_df['Date'].dt.month
# transctional_df['year']=transctional_df['Date'].dt.year
transctional_df['time'] = transctional_df['CheckTime'].dt.time

# Concatenate 'StoreID', 'Date', and 'CheckTime' to create 'Key'
transctional_df['Key'] = transctional_df['StoreId'].astype(str) + "_" + transctional_df['date'].astype(str) + "_" + transctional_df['time'].astype(str)



/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_50489/1297646031.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  transctional_df[datetime_col] = transctional_df[datetime_col].apply(pd.to_datetime, errors='coerce')


In [12]:
transctional_df


,StoreId,Date,CheckTime,UniqueCheckId,PLU,MenuItemId,MenuItemName,ItemCount,Amount,Tax,...,order_channel,Order_type,discount_amt,DiscountName,date,hour,month,year,time,Key
0,1051.0,2024-01-03,2025-02-26 19:04:00,324010510,49864.0,1212.0,No Thanks,1.0,0.00,0.00,...,Dine In - Kiosk\r,Kiosk,NaN,NaN,2024-01-03,0.0,1.0,2024.0,19:04:00,1051.0_2024-01-03_19:04:00
1,1051.0,2024-01-03,2025-02-26 19:04:00,324010510,49022.0,213.0,Large Potato Oles,1.0,1.09,0.09,...,Dine In - Kiosk\r,Kiosk,NaN,NaN,2024-01-03,0.0,1.0,2024.0,19:04:00,1051.0_2024-01-03_19:04:00
2,1051.0,2024-01-03,2025-02-26 19:04:00,324010510,62240.0,913.0,Nacho Crunch Chicken Burrito,1.0,3.00,0.25,...,Dine In - Kiosk\r,Kiosk,NaN,NaN,2024-01-03,0.0,1.0,2024.0,19:04:00,1051.0_2024-01-03_19:04:00
3,1051.0,2024-01-03,2025-02-26 19:04:00,324010510,64505.0,403.0,4 Cheese Chicken Quesadilla,1.0,4.69,0.38,...,Dine In - Kiosk\r,Kiosk,NaN,NaN,2024-01-03,0.0,1.0,2024.0,19:04:00,1051.0_2024-01-03_19:04:00
4,1051.0,2024-01-03,2025-02-26 19:04:00,324010510,50996.0,1178.0,Street Taco - Ck Trio Combo - Qu,1.0,9.59,0.78,...,Dine In - Kiosk\r,Kiosk,NaN,NaN,2024-01-03,0.0,1.0,2024.0,19:04:00,1051.0_2024-01-03_19:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8448878,1052.0,2024-12-31,2025-02-26 17:15:00,3662401052231510926,49100.0,425.0,Side - Guacamole,1.0,1.39,0.12,...,Third Party Delivery - OLO\r,Online,NaN,NaN,2024-12-31,0.0,12.0,2024.0,17:15:00,1052.0_2024-12-31_17:15:00
8448879,1052.0,2024-12-31,2025-02-26 17:15:00,3662401052231510926,64506.0,404.0,4 Cheese Steak Quesadilla,2.0,11.58,1.04,...,Third Party Delivery - OLO\r,Online,NaN,NaN,2024-12-31,0.0,12.0,2024.0,17:15:00,1052.0_2024-12-31_17:15:00
8448880,1052.0,2024-12-31,2025-02-26 17:19:00,3662401052231917348,NaN,1742.0,Service Charge,1.0,3.52,0.00,...,Third Party Delivery - Grubhub\r,Delivery,NaN,NaN,2024-12-31,0.0,12.0,2024.0,17:19:00,1052.0_2024-12-31_17:19:00
8448881,1052.0,2024-12-31,2025-02-26 17:19:00,3662401052231917348,51842.0,269.0,2 Medium Potato Oles,1.0,0.00,0.00,...,Third Party Delivery - Grubhub\r,Delivery,NaN,NaN,2024-12-31,0.0,12.0,2024.0,17:19:00,1052.0_2024-12-31_17:19:00


In [ ]:
# TRANSACTIONAL DATA PRE_PROCESS

datetime_col=['Date','CheckTime']
float_col=['StoreId','UniqueCheckId','PLU']


transctional_df['date']=transctional_df['Date'].dt.date
transctional_df['time'] = transctional_df['CheckTime'].dt.time


# Concatenate 'StoreID', 'Date', and 'CheckTime' to create 'Key'
transctional_df['Key'] = transctional_df['StoreId'].astype(str) + "_" + transctional_df['date'].astype(str) + "_" + transctional_df['time'].astype(str)
summary_trans=transctional_df.groupby(['Key', 'UniqueCheckId'])['PLU'].unique().reset_index()



In [7]:
transctional_df.head(2)


,StoreId,Date,CheckTime,UniqueCheckId,PLU,MenuItemId,MenuItemName,ItemCount,Amount,Tax,LineType,LineTypeDescription,CategoryName,ReductionId,ReductionName,order_channel,Order_type,discount_amt,DiscountName
0,1051.0,2024-01-03,19:04:00.0000000,324010510,49864,1212.0,No Thanks,1.0,0.00,0.00,1.0,Sale,Misc,0.0,NaN,Dine In - Kiosk\r,Kiosk,NaN,NaN
1,1051.0,2024-01-03,19:04:00.0000000,324010510,49022,213.0,Large Potato Oles,1.0,1.09,0.09,1.0,Sale,Potato Olés,0.0,NaN,Dine In - Kiosk\r,Kiosk,NaN,NaN


In [9]:
summary_trans=transctional_df.groupby(['Key', 'UniqueCheckId'])['PLU'].unique().reset_index()

summary_trans


,Key,UniqueCheckId,PLU
0,1001.0_2023-01-01_09:57:00,123010012000,"[51892.0, 51891.0, 49098.0, 50420.0]"
1,1001.0_2023-01-01_10:01:00,123010014000,"[50064.0, 47478.0]"
2,1001.0_2023-01-01_10:02:00,123010014001,"[49016.0, 49762.0, 47478.0, 49761.0, 50064.0]"
3,1001.0_2023-01-01_10:04:00,123010014002,"[51892.0, 47478.0, 52071.0]"
4,1001.0_2023-01-01_10:05:00,123010013000,"[52061.0, 51891.0, 50396.0, 49083.0, 52588.0]"
...,...,...,...
1817388,nan_NaT_NaT,3662401052140329385,[nan]
1817389,nan_NaT_NaT,3662401052160209971,[nan]
1817390,nan_NaT_NaT,3662401052174231602,[nan]
1817391,nan_NaT_NaT,3662401052175922941,[nan]


In [37]:
# Reduction of datasets
#filter keys in loyalty tble that are present in transactionl df, this way we reduce search for non existed keys.=list(set(receipt_item_summary['Key'].unique()) & set(transctional_df['Key'].unique()))
#step1: find intersaction of keys bwtween transaction and loyal
loyalty_keys=list(set(summary_loyal['Key'].unique()) & set(summary_trans['Key'].unique()))

# #step 2: keep only keys that are in transation
summary_loyal_filtered=summary_loyal[summary_loyal['Key'].isin(loyalty_keys)]

# step 3: check the unique keys
print('initil no of keys',summary_loyal['Key'].nunique())
print('number of common keys',summary_loyal_filtered['Key'].nunique())

print('within common keys, no of unique Checkin ID in loyalty table are:', summary_loyal['Checkin ID'].nunique())
print('within common keys, no of unique UniqueCheckId in transactional table are:', summary_trans[summary_trans['Key'].isin(loyalty_keys)]['UniqueCheckId'].nunique())



initil no of keys 93573
number of common keys 83017
within common keys, no of unique Checkin ID in loyalty table are: 94892
within common keys, no of unique UniqueCheckId in transactional table are: 105680


In [38]:
# finding keys that are not present in transactional df
A = set(loyalty_df['Key'].unique())
B = set(transctional_df['Key'].unique())

# Compute the difference
difference = list(A - B)




In [39]:
receipt_item_summary=summary_loyal_filtered.copy()
receipt_summary_trans=summary_trans.copy()
trans_df = transctional_df.copy()  
trans_df['loyal-member']= '-'


In [27]:
# 1key 2 unique 1001.0_2023-07-13_11:51:00
# 1 key 1 uni 1006.0_2023-02-05_15:38:00
# no key 856008614 1003.0_2023-01-01_10:22:00
# not loyal 1006.0_2023-01-05_09:31:00
# 1 key 2 unique_id one is yes other is no  1005.0_2024-04-10_13:35:00

# groupl=loyalty_df[loyalty_df['Key']== '1005.0_2024-04-10_13:35:00']
# receipt_item_summary=groupl.groupby(['Key', 'Checkin ID'])['Menu Item ID'].unique().reset_index()
# receipt_item_summary


,Key,Checkin ID,Menu Item ID
0,1005.0_2024-04-10_13:35:00,1358440598,"[51893.0, 52075.0]"


In [40]:
# group =transctional_df[transctional_df['Key']== '1005.0_2024-04-10_13:35:00']
# receipt_summary_trans=group.groupby(['Key', 'UniqueCheckId'])['PLU'].unique().reset_index()
# receipt_summary_trans


In [41]:

def loyal_member_taggibng(receipt_item_summary):
    keys_not_in_trans=[]
    total_rows = len(receipt_item_summary)  # Get total number of rows
   
    print('w')
    for index, (i, row) in enumerate(receipt_item_summary.iterrows(), start=1):

        #step one: take first key form loyalty summary
        loyalty_key = row['Key']
        loyalty_ids_list=row['Menu Item ID']
        
        # step 2: locate in transactionnal summart
        trans_group = receipt_summary_trans[receipt_summary_trans['Key'] == loyalty_key]
        tran_unique_ids= trans_group['UniqueCheckId'].nunique()

        # check for cases
        # case 1: no loyalty key found in transaction
        if (tran_unique_ids== 0):
            print('No key found in transactional table')
            keys_not_in_trans.append(loyalty_key)
            trans_df['loyal-member']=np.where((trans_df['Key']==loyalty_key), 'no key found',trans_df['loyal-member'] )
        
            
        elif tran_unique_ids == 1:
            # Flatten PLU column to get all unique item IDs in a single set
            trans_ids_list = set(trans_group['PLU'].explode().dropna())

            if set(loyalty_ids_list).issubset(trans_ids_list):
                print('one key')
                
                # Extract first (or only) unique UniqueCheckId
                unique_id = trans_group['UniqueCheckId'].unique()[0]  # Get scalar value

                # Update 'loyal-member' column in transactional df
                trans_df['loyal-member'] = np.where(
                    (trans_df['Key'] == loyalty_key) & (trans_df['UniqueCheckId'] == unique_id),
                    'Yes',
                    'No'
                )

            
        elif(tran_unique_ids>1):
            print('two key')
            for index, (j, g_row) in enumerate(trans_group.iterrows(), start=1):
                
                grouped_trans_list=set(g_row['PLU'])
                grouped_unique_id=g_row['UniqueCheckId']
                # grouped_trans_list = g_row['PLU'] if isinstance(g_row['PLU'], (list, set)) else []

                if(set(loyalty_ids_list).issubset(set(grouped_trans_list))):
                    trans_df['loyal-member'] = np.where(
                        (trans_df['Key'] == loyalty_key) & (trans_df['UniqueCheckId'] == grouped_unique_id),
                        'Yes',
                         'No'
                    ) 
            
               
        else:
            trans_df['loyal-member'] = np.where(
                (trans_df['Key'] == loyalty_key) ,
                'No',
                trans_df['loyal-member']
            )
            print('Key exists but no simialr items found')
    
        print(f"Processing row {i+1}/{total_rows} ({(i+1)/total_rows:.2%} done)")

    print("Processing complete!")
    return keys_not_in_trans,trans_df


In [44]:
889/60


14.816666666666666

In [43]:
receipt_item_summary=summary_loyal_filtered.copy()
receipt_summary_trans=summary_trans.copy()
trans_df = transctional_df.copy()  
trans_df['loyal-member']= '-'

receipt_item_summary1=receipt_item_summary[:50000].reset_index(drop=True)
receipt_item_summary1.index += 1 


keys_not_in_trans,trans_df1=loyal_member_taggibng(receipt_item_summary1)



w
Processing row 2/50000 (0.00% done)
two key
Processing row 3/50000 (0.01% done)
two key
Processing row 4/50000 (0.01% done)
Processing row 5/50000 (0.01% done)
Processing row 6/50000 (0.01% done)
two key
Processing row 7/50000 (0.01% done)
Processing row 8/50000 (0.02% done)
Processing row 9/50000 (0.02% done)
Processing row 10/50000 (0.02% done)
Processing row 11/50000 (0.02% done)
Processing row 12/50000 (0.02% done)
two key
Processing row 13/50000 (0.03% done)
Processing row 14/50000 (0.03% done)
Processing row 15/50000 (0.03% done)
Processing row 16/50000 (0.03% done)
two key
Processing row 17/50000 (0.03% done)
Processing row 18/50000 (0.04% done)
Processing row 19/50000 (0.04% done)
Processing row 20/50000 (0.04% done)
Processing row 21/50000 (0.04% done)
two key
Processing row 22/50000 (0.04% done)
Processing row 23/50000 (0.05% done)
Processing row 24/50000 (0.05% done)
Processing row 25/50000 (0.05% done)
Processing row 26/50000 (0.05% done)
Processing row 27/50000 (0.05% do

KeyboardInterrupt: 

In [30]:
trans_df1_group =trans_df1[trans_df1['Key']== '1005.0_2024-04-10_13:35:00']
trans_df1_group=trans_df1_group.groupby(['Key', 'UniqueCheckId','loyal-member'])['PLU'].unique().reset_index()
trans_df1_group


,Key,UniqueCheckId,loyal-member,PLU
0,1005.0_2024-04-10_13:35:00,10124010052040,-,"[58843.0, 59388.0]"
1,1005.0_2024-04-10_13:35:00,10124010053191,Yes,"[52075.0, 51893.0]"


In [ ]:

trans_df2=trans_df[trans_df['loyal-member'].isin(['Yes','No']+keys_not_in_trans)]
print(trans_df2.shape)


In [ ]:

with open('output1.csv', 'a', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(trans_df2.columns)  # Write header
    for index, row in trans_df2.iterrows():
        writer.writerow(row.astype(str))  # Convert row to string before writing


In [19]:
# import csv
# # for i in [50000,60000,70000,80000],:
# #     for j in [60000,70000,80000,-1]:
# for i in [50000,50005,50010,50020],:
#     for j in [50005,50010,50020,50025]:
#         receipt_item_summary=summary_loyal_filtered.copy()
#         receipt_summary_trans=summary_trans.copy()
#         trans_df = transctional_df.copy()  
#         trans_df['loyal-member']= '-'

#         receipt_item_summary1=receipt_item_summary[i:j].reset_index(drop=True)
#         receipt_item_summary1.index += 1 

#         keys_not_in_trans,trans_df1=loyal_member_taggibng(receipt_item_summary1)
        
#         trans_df2=trans_df[trans_df['loyal-member'].isin(['Yes','No']+keys_not_in_trans)]
#         print(trans_df2.shape)
        
#         # with open('output1.csv', 'a', newline='') as f:
#         #     writer = csv.writer(f)
#         #     writer.writerow(trans_df2.columns)  # Write header
#         #     for index, row in trans_df2.iterrows():
#         #         writer.writerow(row.astype(str))  # Convert row to string before writing



In [20]:
# keys_not_in_trans


In [ ]:
# trans_df.shape


In [ ]:

trans_df2=trans_df[trans_df['loyal-member'].isin(['Yes','No']+keys_not_in_trans)]
trans_df2.shape



In [ ]:


# import csv

# with open('output1.csv', 'a', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(trans_df2.columns)  # Write header
#     for index, row in trans_df2.iterrows():
#         writer.writerow(row.astype(str))  # Convert row to string before writing


In [28]:
input=pd.read_csv('output1.csv')
input.shape
input.columns


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_72119/3703747728.py:1: DtypeWarning: Columns (0,3,4,5,7,8,9,10,13,17,18,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  input=pd.read_csv('output1.csv')


Index(['StoreId', 'Date', 'CheckTime', 'UniqueCheckId', 'PLU', 'MenuItemId',
       'MenuItemName', 'ItemCount', 'Amount', 'Tax', 'LineType',
       'LineTypeDescription', 'CategoryName', 'ReductionId', 'ReductionName',
       'order_channel', 'Order_type', 'discount_amt', 'DiscountName', 'date',
       'time', 'hour', 'month', 'year', 'Key', 'loyal-member'],
      dtype='object')

In [34]:
transctional_df.columns
transctional_df.shape



(8448883, 25)

In [40]:
duplicates = input[input.duplicated(subset=['Key',  'UniqueCheckId'], keep=False)]
duplicates


,StoreId,Date,CheckTime,UniqueCheckId,PLU,MenuItemId,MenuItemName,ItemCount,Amount,Tax,...,Order_type,discount_amt,DiscountName,date,time,hour,month,year,Key,loyal-member
0,1001.0,2023-07-13 00:00:00,2025-02-11 10:55:00,19423010012023,51891.0,382.0,"Small, Drink",1.0,2.49,0.15,...,Dine,NaN,NaN,2023-07-13,10:55:00,0.0,7.0,2023.0,1001.0_2023-07-13_10:55:00,Yes
1,1001.0,2023-07-13 00:00:00,2025-02-11 10:55:00,19423010012023,50168.0,205.0,Super Nachos - Beef/Bean,1.0,6.79,0.41,...,Dine,NaN,NaN,2023-07-13,10:55:00,0.0,7.0,2023.0,1001.0_2023-07-13_10:55:00,Yes
2,1001.0,2023-07-13 00:00:00,2025-02-11 10:55:00,19423010012023,52055.0,1180.0,Side Ingredients,1.0,0.0,0.0,...,Dine,NaN,NaN,2023-07-13,10:55:00,0.0,7.0,2023.0,1001.0_2023-07-13_10:55:00,Yes
3,1001.0,2023-07-13 00:00:00,2025-02-11 10:55:00,19423010012023,51893.0,384.0,"Large, Drink",1.0,2.99,0.18,...,Dine,NaN,NaN,2023-07-13,10:55:00,0.0,7.0,2023.0,1001.0_2023-07-13_10:55:00,Yes
4,1001.0,2023-07-13 00:00:00,2025-02-11 10:55:00,19423010012023,50352.0,281.0,Softshell Taco - Beef,1.0,2.19,0.13,...,Dine,NaN,NaN,2023-07-13,10:55:00,0.0,7.0,2023.0,1001.0_2023-07-13_10:55:00,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307168,1006.0,2024-11-01 00:00:00,2025-02-12 09:28:00,3062401006152837861,49756.0,77.0,Grilled,1.0,0.0,0.0,...,Online,NaN,NaN,2024-11-01,09:28:00,0.0,11.0,2024.0,1006.0_2024-11-01_09:28:00,Yes
307169,1006.0,2024-11-01 00:00:00,2025-02-12 09:28:00,3062401006152837861,47478.0,18.0,Meat & Potato Breakfast Burrito - Bacon,1.0,4.19,0.12,...,Online,NaN,NaN,2024-11-01,09:28:00,0.0,11.0,2024.0,1006.0_2024-11-01_09:28:00,Yes
307170,1052.0,2024-12-06 00:00:00,2025-02-12 13:38:00,3412401052193800142,49083.0,1926.0,Side Nacho Cheese - Qu,1.0,1.39,0.11,...,Online,NaN,NaN,2024-12-06,13:38:00,0.0,12.0,2024.0,1052.0_2024-12-06_13:38:00,Yes
307171,1052.0,2024-12-06 00:00:00,2025-02-12 13:38:00,3412401052193800142,49132.0,307.0,Spicy Steak & Potato Griller,1.0,4.0,0.32,...,Online,NaN,NaN,2024-12-06,13:38:00,0.0,12.0,2024.0,1052.0_2024-12-06_13:38:00,Yes


In [70]:

list(input['Key'].unique())


62618

In [ ]:
transctional_df


In [90]:
df=transctional_df[~transctional_df['Key'].isin(list(input['Key']))]


In [ ]:
df1=transctional_df[transctional_df['Key'].isin(list(input['Key'].unique()))]



In [78]:
df['loyal_member']='-'


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_72119/680159686.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loyal_member']='-'


In [91]:
merge_df=pd.concat([df,input],axis=1)


In [96]:
merge_df.columns


Index(['StoreId', 'Date', 'CheckTime', 'UniqueCheckId', 'PLU', 'MenuItemId',
       'MenuItemName', 'ItemCount', 'Amount', 'Tax', 'LineType',
       'LineTypeDescription', 'CategoryName', 'ReductionId', 'ReductionName',
       'order_channel', 'Order_type', 'discount_amt', 'DiscountName', 'date',
       'time', 'hour', 'month', 'year', 'Key', 'StoreId', 'Date', 'CheckTime',
       'UniqueCheckId', 'PLU', 'MenuItemId', 'MenuItemName', 'ItemCount',
       'Amount', 'Tax', 'LineType', 'LineTypeDescription', 'CategoryName',
       'ReductionId', 'ReductionName', 'order_channel', 'Order_type',
       'discount_amt', 'DiscountName', 'date', 'time', 'hour', 'month', 'year',
       'Key', 'loyal-member'],
      dtype='object')

In [102]:
merge_df[merge_df['loyal-member']=='Yes']


,StoreId,Date,CheckTime,UniqueCheckId,PLU,MenuItemId,MenuItemName,ItemCount,Amount,Tax,...,Order_type,discount_amt,DiscountName,date,time,hour,month,year,Key,loyal-member
0,1051.0,2024-01-03,2025-02-12 19:04:00,324010510.0,49864.0,1212.0,No Thanks,1.0,0.00,0.00,...,Dine,NaN,NaN,2023-07-13,10:55:00,0.0,7.0,2023.0,1001.0_2023-07-13_10:55:00,Yes
1,1051.0,2024-01-03,2025-02-12 19:04:00,324010510.0,49022.0,213.0,Large Potato Oles,1.0,1.09,0.09,...,Dine,NaN,NaN,2023-07-13,10:55:00,0.0,7.0,2023.0,1001.0_2023-07-13_10:55:00,Yes
2,1051.0,2024-01-03,2025-02-12 19:04:00,324010510.0,62240.0,913.0,Nacho Crunch Chicken Burrito,1.0,3.00,0.25,...,Dine,NaN,NaN,2023-07-13,10:55:00,0.0,7.0,2023.0,1001.0_2023-07-13_10:55:00,Yes
3,1051.0,2024-01-03,2025-02-12 19:04:00,324010510.0,64505.0,403.0,4 Cheese Chicken Quesadilla,1.0,4.69,0.38,...,Dine,NaN,NaN,2023-07-13,10:55:00,0.0,7.0,2023.0,1001.0_2023-07-13_10:55:00,Yes
4,1051.0,2024-01-03,2025-02-12 19:04:00,324010510.0,50996.0,1178.0,Street Taco - Ck Trio Combo - Qu,1.0,9.59,0.78,...,Dine,NaN,NaN,2023-07-13,10:55:00,0.0,7.0,2023.0,1001.0_2023-07-13_10:55:00,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306905,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Take,NaN,NaN,2024-12-31,10:09:00,0.0,12.0,2024.0,1006.0_2024-12-31_10:09:00,Yes
306906,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Dine,NaN,NaN,2024-12-31,11:26:00,0.0,12.0,2024.0,1006.0_2024-12-31_11:26:00,Yes
306907,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Dine,NaN,NaN,2024-12-31,11:26:00,0.0,12.0,2024.0,1006.0_2024-12-31_11:26:00,Yes
306908,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Dine,NaN,NaN,2024-12-31,11:26:00,0.0,12.0,2024.0,1006.0_2024-12-31_11:26:00,Yes


In [92]:
merge_df.shape


(8078777, 51)

370106

In [93]:
transctional_df.shape


(8448883, 25)

In [87]:
merge_df.shape


(8078777, 52)

In [82]:
difference


['1005.0_2023-01-27_17:19:00',
 '1003.0_2023-02-16_10:40:00',
 '1001.0_2023-01-29_12:15:00',
 '1003.0_2023-08-17_15:53:00',
 '1005.0_2024-12-14_22:25:00',
 '1003.0_2023-04-01_14:02:00',
 '1001.0_2024-11-27_13:20:00',
 '1003.0_2023-09-06_10:32:00',
 '1003.0_2023-05-12_13:24:00',
 '1005.0_2024-12-18_10:17:00',
 '1003.0_2023-01-07_11:18:00',
 '1003.0_2023-06-05_11:12:00',
 '1052.0_2024-12-10_11:49:00',
 '1003.0_2023-05-04_19:40:00',
 '1051.0_2024-09-10_16:29:00',
 '1003.0_2024-11-01_10:44:00',
 '1006.0_2024-11-06_17:37:00',
 '1003.0_2023-02-06_19:33:00',
 '1003.0_2023-05-17_10:25:00',
 '1003.0_2023-02-08_09:09:00',
 '1001.0_2023-05-02_10:46:00',
 '1003.0_2023-05-30_10:29:00',
 '1003.0_2023-08-22_09:01:00',
 '1003.0_2023-10-10_16:20:00',
 '1003.0_2023-09-02_12:46:00',
 '1003.0_2023-01-20_11:08:00',
 '1003.0_2023-01-29_15:55:00',
 '1003.0_2023-07-29_17:02:00',
 '1052.0_2024-11-15_14:05:00',
 '1003.0_2023-05-31_19:39:00',
 '1003.0_2023-08-19_13:30:00',
 '1003.0_2023-02-25_18:41:00',
 '1008.0

In [84]:
transctional_df[transctional_df['Key']=='1003.0_2023-05-30_10:29:00']


,StoreId,Date,CheckTime,UniqueCheckId,PLU,MenuItemId,MenuItemName,ItemCount,Amount,Tax,...,order_channel,Order_type,discount_amt,DiscountName,date,time,hour,month,year,Key


In [80]:
df['loyal_member']=np.where(df['Key'].isin(difference),'key not found',df['loyal_member'])


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_72119/634423627.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loyal_member']=np.where(df['Key'].isin(difference),'key not found',df['loyal_member'])


In [81]:
df['loyal_member'].unique()


array(['-'], dtype=object)

In [36]:
df1 = input.drop_duplicates(subset=['Key',  'UniqueCheckId'])


In [37]:
# Merge df1 into df2 on 'key' and 'unique_id'
merged = transctional_df.merge(df1[['Key',  'UniqueCheckId', 'loyal-member']], on=['Key',  'UniqueCheckId'], how='left')


In [57]:
merged['UniqueCheckId'].nunique()

difference


['1005.0_2023-01-27_17:19:00',
 '1003.0_2023-02-16_10:40:00',
 '1001.0_2023-01-29_12:15:00',
 '1003.0_2023-08-17_15:53:00',
 '1005.0_2024-12-14_22:25:00',
 '1003.0_2023-04-01_14:02:00',
 '1001.0_2024-11-27_13:20:00',
 '1003.0_2023-09-06_10:32:00',
 '1003.0_2023-05-12_13:24:00',
 '1005.0_2024-12-18_10:17:00',
 '1003.0_2023-01-07_11:18:00',
 '1003.0_2023-06-05_11:12:00',
 '1052.0_2024-12-10_11:49:00',
 '1003.0_2023-05-04_19:40:00',
 '1051.0_2024-09-10_16:29:00',
 '1003.0_2024-11-01_10:44:00',
 '1006.0_2024-11-06_17:37:00',
 '1003.0_2023-02-06_19:33:00',
 '1003.0_2023-05-17_10:25:00',
 '1003.0_2023-02-08_09:09:00',
 '1001.0_2023-05-02_10:46:00',
 '1003.0_2023-05-30_10:29:00',
 '1003.0_2023-08-22_09:01:00',
 '1003.0_2023-10-10_16:20:00',
 '1003.0_2023-09-02_12:46:00',
 '1003.0_2023-01-20_11:08:00',
 '1003.0_2023-01-29_15:55:00',
 '1003.0_2023-07-29_17:02:00',
 '1052.0_2024-11-15_14:05:00',
 '1003.0_2023-05-31_19:39:00',
 '1003.0_2023-08-19_13:30:00',
 '1003.0_2023-02-25_18:41:00',
 '1008.0

In [55]:

merged['loyal-member']=np.where(merged['Key'].isin(difference),'Key not found',merged['loyal-member'])


In [58]:
merged[merged['Key']=='1005.0_2023-01-27_17:19:00']


,StoreId,Date,CheckTime,UniqueCheckId,PLU,MenuItemId,MenuItemName,ItemCount,Amount,Tax,...,Order_type,discount_amt,DiscountName,date,time,hour,month,year,Key,loyal-member


In [59]:
merged['loyal-member'].unique()


array([nan, 'Yes'], dtype=object)

In [62]:
merged[merged['loyal-member']=='Yes']["Key"].nunique()
(28908/84000)*100


34.41428571428571

In [ ]:
merged[merged['loyal-member']=='Yes']["Key"].nunique()


In [22]:
input['loyal-member'].unique()


array(['Yes', 'loyal-member'], dtype=object)

In [ ]:
# receipt_summary_trans[receipt_summary_trans['Key']=='1001.0_2023-07-13_10:55:00']


In [ ]:
# receipt_item_summary[receipt_item_summary['Key']=='1001.0_2023-07-13_10:55:00']
